In [1]:
!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets

In [1]:
!pip install -q ipywidgets

In [7]:
!pip install -q pypdf

In [1]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

In [5]:
import os

In [6]:
# Import the load_dotenv function from the dotenv module
from dotenv import load_dotenv

# Call the load_dotenv function to load environment variables from a .env file
load_dotenv()

os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [26]:
# ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [54]:
import glob

In [62]:
pdfs = []
for file in glob.glob("/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data" + "/*.pdf"):
    print(file)
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    pdfs.extend(pages)

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/exhibit-13.pdf
/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/third-quarter-10q.pdf
/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/second-quarter-10q.pdf


In [16]:
pdfs[0]

Document(page_content='Exhibit 13 \nFinancial Review \n4 Overview 110 4 Equity Securities \n9 Earnings Performance 112 5 Loans and Related Allowance for Credit Losses \n27 Balance Sheet Analysis 127 6 Mortgage Banking Activities \n29 Off-Balance Sheet Arrangements 129 7 Intangible Assets and Other Assets \n30 Risk Management 130 8 Leasing Activity \n52 Capital Management 132 9 Deposits \n58 Regulatory Matters 133 10 Long-Term Debt \n61 Critical Accounting Policies 135 11 Preferred Stock \n65 Current Accounting Developments 136 12 Common Stock and Stock Plans \n66 Forward-Looking Statements 138 13 Legal Actions \n68 Risk Factors 140 14 Derivatives \nControls and Procedures 146 15 Fair Values of Assets and Liabilities \n82 Disclosure Controls and Procedures 157 16 Securitizations and Variable Interest Entities \n82 Internal Control Over Financial Reporting 162 17 Guarantees and Other Commitments \n82 Management’s Report on Internal Control over \nFinancial Reporting 165 18 Securities and

In [17]:
pdfs[0].metadata

{'source': 'data/exhibit-13.pdf', 'page': 0}

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain.docstore.document import Document as LangchainDocument

In [49]:
RAW_KNOWLEDGE_BASE = [LangchainDocument(page_content=page.page_content, metadata=page.metadata) for page in tqdm(pdfs)]

  0%|          | 0/903 [00:00<?, ?it/s]

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=page.page_content, metadata=page.metadata) for page in tqdm(pdfs)]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])


  0%|          | 0/903 [00:00<?, ?it/s]

In [19]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [20]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [30]:
import random

N_GENERATIONS = 20  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 20 QA couples...


  0%|          | 0/20 [00:00<?, ?it/s]

In [31]:
display(pd.DataFrame(outputs).tail(10))

,context,question,answer,source_doc
10,"impact of the Update to our income tax disclosures. Upon adoption, those disclosures may change as \nfollows: \n• For the tabular effective income tax rate reconciliation, alignment to specific categories (where \napplicable) and further disaggregation of certain categories (where applicable) by nature and/or \njurisdiction if the reconciling item is 5% or more of the statutory tax expense. \n• Description of states and loca l jurisdictions that contribute the majority of the effect of the state \nand local income tax category of the effective income tax rate reconciliation. \n• Disaggregate the amount of income ta xes paid (net of refunds) by federal, state, and non-U.S. taxes \nand further disaggregate by individual jurisdictions where income taxes paid (net of refunds) is 5% \nor more of total income taxes paid (net of refunds). \n• Disaggregate net income (or loss) before income ta x expense (or benefit) between domestic and \nnon-U.S. \nOther Accounting Developments \nThe following Updates are applicable to us but are not expected \nto have a material impact on our consolidated financial \nstatements: \n• ASU 2022-03 – Fair Value Measurement (Topic 820): Fair \nValue Measurement of Equity Securities Subject to Contractual \nSale Restrictions \n• ASU 2023-08 - Intangibles - Goodwill and Other - Crypto \nAssets (Subtopic 350-60): Accounting for and Disclosure of \nCrypto Assets \nWells Fargo & Company 65",What is the name of the ASU that is not expected to have a material impact on the consolidated financial statements?\n,ASU 2022-03 – Fair Value Measurement (Topic 820): Fair Value Measurement of Equity Securities Subject to Contractual Sale Restrictions and ASU 2023-08 - Intangibles - Goodwill and Other - Crypto Assets (Subtopic 350-60): Accounting for and Disclosure of Crypto Assets,data/exhibit-13.pdf
11,"caution should be exercised when relying on this data. Changes in \nvalue based on variations in assumptions generally cannot be \nextrapolated because the relationship of the change in the \nassumption to the change in value may not be linear. Also, the \neffect of a variation in a particular assumption on the value of the other interests held is calculated independently without changing \nany other assumptions. In reality, changes in one factor may",How are changes in assumptions calculated in relation to the value of other interests held?\n,Changes in assumptions are calculated independently without changing any other assumptions.,data/exhibit-13.pdf
12,"Table 9f: Corporate and Investment Banking – Balance Sheet \nYear ended December 31, \n($ in millions) 2023 2022 $ Change \n2023/ \n2022 % Change \n2023/ \n2022 2021 $ Change \n2022/ \n2021 % Change \n2022/ \n2021 \nSelected Balance Sheet Data (average) \nLoans: \nCommercial and industrial $ 191,602 198,424 (6,822) (3)% $ 170,713 27,711 16 % \nCommercial real estate 100,373 98,560 1,813 2 86,323 12,237 14 \nTotal loans $ 291,975 296,984 (5,009) (2) $ 257,036 39,948 16 \nLoans by Line of Business: \nBanking $ 95,783 106,440 (10,657) (10) $ 93,766 12,674 14 \nCommercial Real Estate 135,702 133,719 1,983 1 110,978 22,741 20 \nMarkets 60,490 56,825 3,665 6 52,292 4,533 9 \nTotal loans $ 291,975 296,984 (5,009) (2) $ 257,036 39,948 16 \nTrading-related assets: \nTrading account securities $ 118,130 112,213 5,917 5 $ 110,386 1,827 2 \nReverse repurchase agreements/securities borrowed 61,510 50,491 11,019 22 59,044 (8,553) (14) \nDerivative assets 18,636 27,421 (8,785) (32) 25,315 2,106 8 \nTotal trading-related assets $ 198,276 190,125 8,151 4 $ 194,745 (4,620) (2) \nTotal assets 553,722 557,396 (3,674) (1) 523,344 34,052 7 \nTotal deposits 162,062 161,720 342 — 189,176 (27,456) (15) \nAllocated capital 44,000 36,000 8,000 22 34,000 2,000 6 \nSelected Balance Sheet Data (period-end) \nLoans: \nCommercial and industrial $ 189,379 196,529 (7,150) (4) $ 191,391 5,138 3 \nCommercial real estate 98,053 101,848 (3,795) (4) 92,983 8,865 10 

In [27]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independant from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [32]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/20 [00:00<?, ?it/s]

In [33]:
generated_questions = pd.DataFrame.from_dict(outputs)

In [34]:
print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What was the impact on noninterest income as a result of the adoption of ASU 2018-12?\n,The impact on noninterest income as a result of the adoption of ASU 2018-12 was an increase of $583 million.,5,1,4
1,"How many millions of dollars are in total loans not government insured/guaranteed that are 90 days or more past due and still accruing as of December 31, 2023?\n","1,105 million dollars",5,1,5
2,What did the FRB do to address high inflation?\n,The FRB significantly raised its target range for the federal funds rate over the last two years to address high inflation.,5,2,4
3,What is the total equity of the company at the end of 2023?\n,"The total equity of the company at the end of 2023 is $187,443 million.",1,1,5
4,What is the discount rate for interest rate contracts: derivative loan commitments in the context?\n,The discount rate for interest rate contracts: derivative loan commitments in the context is 4.2%.,1,1,1
5,What is the G-SIB capital surcharge for 2024?\n,The G-SIB capital surcharge will continue to be 1.50% in 2024.,1,1,4
6,What is the minimum amount of TLAC required for a U.S. G-SIB?\n,The minimum amount of TLAC required for a U.S. G-SIB is the greater of 18.00% of RWAs or 7.50% of total leverage exposure plus a TLAC buffer and an external TLAC leverage buffer.,5,1,5
7,What is CVA estimated based on?\n,CVA is estimated based on observed credit spreads in the credit default swap market and indices indicative of the credit quality of the counterparties to our derivatives.,5,4,5
8,What could impact the quality or effectiveness of the data used by the company?\n,"Issues with the quality or effectiveness of the data could be due to problems with the aggregation, management, and validation procedures.",5,5,5
9,What is the G-SIB surcharge for the Company?\n,The G-SIB surcharge for the Company is 1.50%.,1,1,2


In [36]:
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]

In [37]:
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
7,What is CVA estimated based on?\n,CVA is estimated based on observed credit spreads in the credit default swap market and indices indicative of the credit quality of the counterparties to our derivatives.,5,4,5
8,What could impact the quality or effectiveness of the data used by the company?\n,"Issues with the quality or effectiveness of the data could be due to problems with the aggregation, management, and validation procedures.",5,5,5


In [50]:
eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

In [51]:
eval_dataset

Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'groundedness_score', 'groundedness_eval', 'relevance_score', 'relevance_eval', 'standalone_score', 'standalone_eval'],
    num_rows: 2
})

In [38]:
def split_documents(
    chunk_size: int,
    knowledge_base: List,
    tokenizer_name: str,
) -> List:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)
 
    return docs_processed_unique

In [39]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True},  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

In [40]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [41]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [42]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM


def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = llm(final_prompt)

    return answer, relevant_docs

In [44]:
def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(question, llm, knowledge_index, reranker=reranker)
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [45]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [48]:
from langchain.chat_models import ChatOllama

eval_chat_model = ChatOllama(model="llama2", temperature=0)
evaluator_name = "llama2"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [item.strip() for item in eval_result.content.split("[RESULT]")]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

In [52]:
if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            reranker = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0") if rerank else None
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

Running evaluation for chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:zephyr-7b-beta:
Loading knowledge base embeddings...


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

AssertionError: Torch not compiled with CUDA enabled